In [125]:
import os
os.chdir("../src")

import tensorflow as tf
import numpy as np
from dataloader import qm9_parse, qm9_fetch
import dmol


In [126]:
qm9_records = qm9_fetch()

data = qm9_parse(qm9_records)


Found existing record file, delete if you want to re-fetch


In [127]:
# for d in data:
#     print(d)
#     break


In [128]:
#Train with 5000 and use 1000 for test
shuffled_data = data.shuffle(700, reshuffle_each_iteration=False)
test_set = shuffled_data.take(100)
valid_set = shuffled_data.skip(100).take(100)
train_set = shuffled_data.skip(200).take(500)


In [129]:
for d in train_set:
    print(d)
    break


((<tf.Tensor: shape=(9,), dtype=int64, numpy=array([8, 6, 6, 6, 8, 1, 1, 1, 1])>, <tf.Tensor: shape=(9, 4), dtype=float32, numpy=
array([[ 0.04424004,  0.03933243, -0.33071104, -0.308076  ],
       [-0.04854479,  1.1741623 ,  0.00410076,  0.452632  ],
       [-1.1421165 ,  2.2361186 ,  0.15338361, -0.400942  ],
       [ 0.02332538,  3.1235702 ,  0.6168162 , -0.072585  ],
       [ 0.9566381 ,  2.0119011 ,  0.42603746, -0.231459  ],
       [-1.6257375 ,  2.514822  , -0.784011  ,  0.149267  ],
       [-1.9000921 ,  1.9955338 ,  0.9002866 ,  0.149263  ],
       [ 0.28368077,  3.9617276 , -0.03229287,  0.13095   ],
       [ 0.0078225 ,  3.4394267 ,  1.6616273 ,  0.13095   ]],
      dtype=float32)>), <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([ 9.2000000e+01,  1.2446430e+01,  5.2332802e+00,  3.8659899e+00,
        3.9339001e+00,  3.5400002e+01, -2.7880001e-01,  8.8999998e-03,
        2.8780001e-01,  3.3607919e+02,  6.8573996e-02, -2.6710620e+02,
       -2.6710193e+02, -2.6710098e+0

In [130]:
import torch
import torch.nn as nn

# Define the embedding layer outside the loop
embedding_layer = nn.Embedding(16, 128)

# def convert_record(d):
#     # break up record
#     (e, x), y = d
#     e = e.numpy()
#     x= x.numpy()
#     r= x[:,:3]
#     # embedding= nn.Embedding(100,128,padding_idx=0)
#     e=torch.tensor(e)
#     e = embedding(e)
#     r =torch.tensor(r)
#     return(e,r),y.numpy()[13]

# for d in data:
#     print(d)
#     (e, x), y = convert_record(d)
#     print("Embedded Atomic Numbers:\n", e)
#     print("Coordinates:\n", x)
#     print("Label:", y)
#     print()
#     break


In [131]:
def convert_record(d):
    # break up record
    (e, x), y = d
    e = e.numpy()
    x= x.numpy()
    r= x[:,:3]
    embedding= nn.Embedding(100,128,padding_idx=0)
    e=torch.tensor(e)
    s_j = embedding(e)
    r =torch.tensor(r)
    return(s_j,r),y.numpy()[13]


In [132]:
def x2e(x):
    """convert xyz coordinates to inverse pairwise distance"""
    r2 = torch.sum((x - x[:, None, :]) ** 2, axis=-1)
    # r2 = jnp.sum((x - x[:, jnp.newaxis, :]) ** 2, axis=-1)
    # e = jnp.where(r2 != 0, 1 / r2, 0.0)
    e = torch.sqrt(torch.where(r2 != 0, r2, 0.0))

    return e


In [133]:
data_type = train_set
for d in data_type:
    (s_j,r),response_var = convert_record(d)


In [134]:
for atom in r:
    print(atom)


tensor([0.0600, 1.5369, 0.0097])
tensor([ 1.3241,  0.7053, -0.0068])
tensor([-0.0171,  0.0431,  0.0025])
tensor([-0.6194, -0.8931,  1.0450])
tensor([-1.1798, -1.7240,  0.0041])
tensor([-0.6393, -0.8819, -1.0383])
tensor([-0.1994,  2.0575,  0.9265])
tensor([-0.2168,  2.0672, -0.8963])
tensor([ 1.9041,  0.6720, -0.9240])
tensor([1.9215, 0.6622, 0.8988])
tensor([ 0.0969, -1.4442,  1.6694])
tensor([-1.3974, -0.4610,  1.6891])
tensor([-1.4294, -0.4430, -1.6627])
tensor([ 0.0648, -1.4262, -1.6822])


In [135]:
# def create_r_ij(r, r_cut=5.0):
#     r_ij = torch.tensor([])
#     for i in range(len(r)):
#         for j in range(len(r)):
#             if torch.norm(r[i] - r[j]) <r_cut:
#                 r_ij = torch.cat((r_ij, r[i] - r[j])) #important
#             else:
#                 continue
#         return r_ij


In [136]:
# vi bruger denne
def create_r_ij(r, r_cut=5.0):
    r_ij = []
    for i in range(len(r)):
        for j in range(len(r)):
            if torch.norm(r[i] - r[j]) <r_cut:
                r_ij.append(r[i] - r[j]) #important
            else:
                continue
        return r_ij
create_r_ij(r,r_cut=5.0)


[tensor([0., 0., 0.]),
 tensor([-1.2641,  0.8317,  0.0165]),
 tensor([0.0771, 1.4938, 0.0073]),
 tensor([ 0.6794,  2.4300, -1.0352]),
 tensor([1.2398, 3.2609, 0.0057]),
 tensor([0.6993, 2.4188, 1.0481]),
 tensor([ 0.2594, -0.5206, -0.9168]),
 tensor([ 0.2768, -0.5303,  0.9060]),
 tensor([-1.8441,  0.8650,  0.9338]),
 tensor([-1.8615,  0.8748, -0.8890]),
 tensor([-0.0369,  2.9811, -1.6597]),
 tensor([ 1.4574,  1.9980, -1.6793]),
 tensor([1.4894, 1.9800, 1.6725]),
 tensor([-0.0048,  2.9631,  1.6920])]

In [153]:
# define input

r_ij = create_r_ij(r,r_cut=5.0) #vectorization
A = x2e(r) # adjacency matrix
r_pos = r # position matrix
s_j = s_j
r_cut = 5


In [154]:
r_ij_tensor = torch.stack(r_ij)
r_ij_norm  = torch.norm(r_ij_tensor)
v_norm = r_ij_tensor/(torch.norm(r_ij_tensor))
v_j = torch.zeros(128)


In [155]:
r_ij_norm = r_ij_norm.item()
type(r_ij_norm)


float

In [159]:
np.sin(20*np.pi/r_cut)


-4.898587196589413e-16

# Kat tester vores netværk

In [175]:
from typing import Callable, Dict, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F


# Inputs som skal fjernes

#################



__all__ = ["phi", "w", "u", "v", "s", "Message_block", "Update_block", "PaiNN"]

class phi(nn.Module):
    def __init__(self, s_j):
        super().__init__()
        activation_fn = nn.SiLU
        self.net = nn.Sequential(
            nn.Linear(s_j, 128),
            activation_fn(),
            nn.Linear(128, 384),
        )

    def forward(self, x):
        return self.net(x)

# class w(nn.Module):
#     def __init__(self, r_ij_norm, r_cut ):
#         super().__init__()
#         N = [1,2,3,4,5,6,7,8,9,10]
#         RBF = []
#         for n in N:#range(1,21):
#             # if n>0: # hvorfor er denne her???? Hvad er i?
#             RBF.append(np.sin((n*np.pi/r_cut)*r_ij_norm)/r_ij_norm)
#             # RBF.append(torch.sin((n*torch.pi()/r_cut)*r_ij_norm)/r_ij_norm)
#                     # ctx = torch.cat([q, mu_Vn], dim=-1)

#             # else:
#             #     continue
#         f_c=0.5*torch.cos(torch.pi()*r_ij_norm/r_cut)+1
#         fcRBF = RBF * f_c
#         self.fcRBF=nn.linear(fcRBF,384)


import numpy as np
import torch
import torch.nn as nn

class w(nn.Module):
    def __init__(self, r_ij_norm, r_cut):
        super().__init__()
        N = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

        # Convert r_ij_norm to a numpy array if it's not already
        r_ij_norm = np.array(r_ij_norm) if not isinstance(r_ij_norm, np.ndarray) else r_ij_norm

        # Calculate RBF values
        RBF = [np.sin((n * np.pi / r_cut) * r_ij_norm) / r_ij_norm for n in N]

        # Stack RBF values to create a single tensor
        RBF = np.stack(RBF, axis=-1)

        # Calculate f_c
        f_c = 0.5 * np.cos(np.pi * r_ij_norm / r_cut) + 1

        # Ensure f_c is expanded to match RBF dimensions for element-wise multiplication
        f_c = np.expand_dims(f_c, axis=-1)  # Assuming f_c needs to be expanded

        # Element-wise multiplication
        fcRBF = RBF * f_c

        # Convert fcRBF to a torch tensor
        fcRBF = torch.tensor(fcRBF, dtype=torch.float32)

        # Define a linear layer
        self.fcRBF = nn.Linear(fcRBF.size(-1), 384)

    # def forward(self, x):
    #     # Assuming x is the input tensor that you want to process
    #     # Apply the linear transformation
    #     return self.fcRBF(x)


    def forward(self, x):
        rbf_out = self.r_RBF(x)
        return rbf_out * self.f_c





class Message_block(nn.Module):
    def __init__(self, phi_input_size, r_ij_norm, r_cut):
        super().__init__()
        self.phi_layer = phi(phi_input_size)
        self.w_layer = w(r_ij_norm, r_cut)

        # Maybe fungerer det herover

    def forward(self, input1, input2, v_j, s_j, v_norm):
        phi_output = self.phi_layer(input1)
        w_output = self.w_layer(input2)

        ## måske dette i stedet??
        # phi = self.net(input1)
        # w_output = self.net(input2)

        output = phi_output * w_output
        output = torch.split(output, 3)
        output1 = output[0] * v_j
        s_m = torch.sum(output[1]) + s_j
        output3 = output[2] * v_norm
        v_m = torch.sum(output1 + output3) + v_j
        return v_m, s_m


### update block

class u(nn.Module):
    def __init__(self, v_m_size):
        super().__init__()
        self.net = nn.Linear(v_m_size, 128)

    def forward(self, x):
        return self.net(x)

class v(nn.Module):
    def __init__(self, v_m):
        super().__init__()
        self.net = nn.Linear(v_m,128)

class s(nn.Module):
    def __init__(self, v_norm, s_m):
        super().__init__()
        activation_fn = nn.SiLU
        self.net = nn.Sequential(
            # torch.stack(v_norm,s_m),
            torch.stack([v_norm, s_m], dim=0), # nyt # vi er nået hertil
            nn.Linear(256,128),
            activation_fn(),
            nn.Linear(128,384))

class Update_block(nn.Module):
    def __init__(self, v_m_size, s_m_size):
        super().__init__()
        self.u_layer = u(v_m_size)
        self.v_layer = v(v_m_size)
        self.s_layer = s(v_norm, s_m_size)

    def forward(self, input3, input4, input5, v_m, s_m):
        U = self.u_layer(input3)
        V = self.v_layer(input4)
        S = self.s_layer(input5)
        V_dup = V.repeat(128, 2)
        S = torch.split(S, 3)
        output4 = S[0] * U
        output5 = S[1] * V_dup
        output6 = S[2] + output5
        v_u = output4 + v_m
        s_u = output6 + s_m
        return v_u, s_u





class PaiNN(nn.Module):
    def __init__(self, phi_input_size, r_ij, r_cut, v_m_size, s_m_size):
        super().__init__()
        self.message_block = Message_block(phi_input_size, r_ij, r_cut)
        self.update_block = Update_block(v_m_size, s_m_size)


    def forward(self, input1, input2, v_j, s_j, v_norm):
        # Forward pass through the message block
        v_m, s_m = self.message_block(input1, input2, v_j, s_j, v_norm)

        # Forward pass through the update block
        v_u, s_u = self.update_block(v_m, v_m, s_m, v_m, s_m)

        # Return the updated values
        return v_u, s_u


In [176]:
phi_input_size = 128
v_m_size = 128
s_m_size = 128


# input1 = torch.rand(10, phi_input_size)
# input2 = torch.rand(10, r_ij.size(1))
# v_j = torch.rand(128)
# s_j = torch.rand(128)
# v_norm = torch.rand(128)


In [177]:


model = PaiNN(phi_input_size, r_ij, r_cut, v_m_size, s_m_size)


TypeError: expected Tensor as element 1 in argument 0, but got int

In [ ]:
# input1 = torch.rand(10, phi_input_size)
# input2 = torch.rand(10, r_ij.size(1))
# v_j = torch.rand(128)
# s_j = torch.rand(128)
# v_norm = torch.rand(128)


In [ ]:
v_u, s_u = model(input1, input2, v_j, s_j, v_norm)


In [ ]:
# Define a loss function and optimizer
criterion = nn.MSELoss()  # Example loss function, change as needed
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Example optimizer

# Example training loop
for epoch in range(num_epochs):  # num_epochs is the number of epochs
    optimizer.zero_grad()
    outputs = model(input1, input2, v_j, s_j, v_norm)
    loss = criterion(outputs, target)  # target needs to be defined
    loss.backward()
    optimizer.step()


In [192]:

#Message block

v_norm = r_ij_tensor/(torch.norm(r_ij_tensor))

v_j = torch.zeros(128)

class phi(nn.Module):

     def __init__(self):
        super().__init__()
        activation_fn = nn.SiLU

        self.net = nn.Sequential(
            nn.Linear(s_j, 128),
            activation_fn(),
            nn.Linear(128, 384),
        )
class w(nn.Module):
    def __init__(self):
        super().__init__()
        RBF = []
        for n in range(21):
            if i>0:
                RBF.append = torch.sin((n*torch.pi()/r_cut)*r_ij)/r_ij
            else:
                continue
        f_c=0.5*torch.cos(torch.pi()*r_ij/r_cut)+1
        fcRBF = RBF * f_c
        self.fcRBF=nn.linear(fcRBF,384)


def forward(self, input1, input2):
    phi = self.net(input1)
    w = self.net(input2)
    output = phi * w
    output = torch.split(output,3)
    output1 = output[0] * v_j
    s_m = torch.sum(output[1]) + s_j
    output3 = output[2]*v_norm
    v_m= torch.sum(output1+output3) + v_j
    return v_m, s_m


In [193]:
v_m, s_m = forward(self, s_j, r_ij) # input1, input2


NameError: name 'self' is not defined

In [ ]:

#Update block

class u(nn.Module):
      def __init__(self):
        super().__init__()
        self.net = nn.Linear(v_m,128)
class v(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Linear(v_m,128)
class s(nn.Module):
    def __init__(self):
        super().__init__()
        activation_fn = nn.SiLU
        self.net = nn.Sequential(
            torch.stack(v_norm,s_m),
            nn.Linear(256,128),
            activation_fn(),
            nn.Linear(128,384))

def forward(self,input3,input4,input5):
    U = self.net(input3)
    V = self.net(input4)
    S = self.net(input5)
    V_dup = V.repeat(128,2)
    S = torch.split(S,3)
    output4 = S[0]*U
    output5 = S[1]*V_dup
    output6 = S[2]+output5
    v_u = output4 + v_m
    s_u = output6 + s_m
    return v_u, s_u


In [ ]:

#Class pain

#Kør 3x message og 3x update (skiftesvis)

#Linear, SiLU, Linear, output




# class PAINN(nn.Module):
#     def __init__(self, n_basis=int(128)):
#         super(PAINN, self).__init__()
#         self.n_basis = n_basis
#         self.embedding = nn.Embedding(16,n_basis,padding_idx=0)

#     def x2e(r):
#     """convert xyz coordinates to pairwise distance"""
#     r2 = np.sqrt(np.sum((r - r[:, np.newaxis, :]) ** 2, axis=-1))
#     r_cut = 5
#     r_ij = np.where(r2 < r_cut)
#     return r_ij



#     def init_weights(g, n, m):
#     we = np.random.normal(size=(n, m), scale=1e-1)
#     wb = np.random.normal(size=(m), scale=1e-1)
#     wv = np.random.normal(size=(m, n), scale=1e-1)
#     wu = np.random.normal(size=(n, g), scale=1e-1)
#     return [we, wb, wv, wu]


In [ ]:
class update(nn.Module):

class message(nn.Module):

     def __init__(self):
        super().__init__()
        activation_fn = nn.SiLU

        self.net = nn.Sequential(
            nn.Linear(s_j, 128),
            activation_fn(),
            nn.Linear(128, 128),
        )



class PAINN(nn.Module):
    def __init__(self, n_basis=int(128)):
        super(PAINN, self).__init__()
        self.n_basis = n_basis
        self.embedding = nn.Embedding(16,n_basis,padding_idx=0)

    def x2e(r):
    """convert xyz coordinates to pairwise distance"""
    r2 = np.sqrt(np.sum((r - r[:, np.newaxis, :]) ** 2, axis=-1))
    r_cut = 5
    r_ij = np.where(r2 < r_cut)
    return r_ij

    def init_weights(g, n, m):
    we = np.random.normal(size=(n, m), scale=1e-1)
    wb = np.random.normal(size=(m), scale=1e-1)
    wv = np.random.normal(size=(m, n), scale=1e-1)
    wu = np.random.normal(size=(n, g), scale=1e-1)
    return [we, wb, wv, wu]


In [12]:
from typing import Callable, Dict, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F

import schnetpack.properties as properties
import schnetpack.nn as snn

__all__ = ["PaiNN", "PaiNNInteraction", "PaiNNMixing"]


class PaiNNInteraction(nn.Module):
    r"""PaiNN interaction block for modeling equivariant interactions of atomistic systems."""

    def __init__(self, n_atom_basis: int, activation: Callable):
        """
        Args:
            n_atom_basis: number of features to describe atomic environments.
            activation: if None, no activation function is used.
            epsilon: stability constant added in norm to prevent numerical instabilities
        """
        super(PaiNNInteraction, self).__init__()
        self.n_atom_basis = n_atom_basis

        self.interatomic_context_net = nn.Sequential(
            snn.Dense(n_atom_basis, n_atom_basis, activation=activation),
            snn.Dense(n_atom_basis, 3 * n_atom_basis, activation=None),
        )

    def forward(
        self,
        q: torch.Tensor,
        mu: torch.Tensor,
        Wij: torch.Tensor,
        dir_ij: torch.Tensor,
        idx_i: torch.Tensor,
        idx_j: torch.Tensor,
        n_atoms: int,
    ):
        """Compute interaction output.

        Args:
            q: scalar input values
            mu: vector input values
            Wij: filter
            idx_i: index of center atom i
            idx_j: index of neighbors j

        Returns:
            atom features after interaction
        """
        # inter-atomic
        x = self.interatomic_context_net(q)
        xj = x[idx_j]
        muj = mu[idx_j]
        x = Wij * xj

        dq, dmuR, dmumu = torch.split(x, self.n_atom_basis, dim=-1)
        dq = snn.scatter_add(dq, idx_i, dim_size=n_atoms)
        dmu = dmuR * dir_ij[..., None] + dmumu * muj
        dmu = snn.scatter_add(dmu, idx_i, dim_size=n_atoms)

        q = q + dq
        mu = mu + dmu

        return q, mu


class PaiNNMixing(nn.Module):
    r"""PaiNN interaction block for mixing on atom features."""

    def __init__(self, n_atom_basis: int, activation: Callable, epsilon: float = 1e-8):
        """
        Args:
            n_atom_basis: number of features to describe atomic environments.
            activation: if None, no activation function is used.
            epsilon: stability constant added in norm to prevent numerical instabilities
        """
        super(PaiNNMixing, self).__init__()
        self.n_atom_basis = n_atom_basis

        self.intraatomic_context_net = nn.Sequential(
            snn.Dense(2 * n_atom_basis, n_atom_basis, activation=activation),
            snn.Dense(n_atom_basis, 3 * n_atom_basis, activation=None),
        )
        self.mu_channel_mix = snn.Dense(
            n_atom_basis, 2 * n_atom_basis, activation=None, bias=False
        )
        self.epsilon = epsilon

    def forward(self, q: torch.Tensor, mu: torch.Tensor):
        """Compute intraatomic mixing.

        Args:
            q: scalar input values
            mu: vector input values

        Returns:
            atom features after interaction
        """
        ## intra-atomic
        mu_mix = self.mu_channel_mix(mu)
        mu_V, mu_W = torch.split(mu_mix, self.n_atom_basis, dim=-1)
        mu_Vn = torch.sqrt(torch.sum(mu_V**2, dim=-2, keepdim=True) + self.epsilon)

        ctx = torch.cat([q, mu_Vn], dim=-1)
        x = self.intraatomic_context_net(ctx)

        dq_intra, dmu_intra, dqmu_intra = torch.split(x, self.n_atom_basis, dim=-1)
        dmu_intra = dmu_intra * mu_W

        dqmu_intra = dqmu_intra * torch.sum(mu_V * mu_W, dim=1, keepdim=True)

        q = q + dq_intra + dqmu_intra
        mu = mu + dmu_intra
        return q, mu


class PaiNN(nn.Module):
    """PaiNN - polarizable interaction neural network

    References:

    .. [#painn1] Schütt, Unke, Gastegger:
       Equivariant message passing for the prediction of tensorial properties and molecular spectra.
       ICML 2021, http://proceedings.mlr.press/v139/schutt21a.html

    """

    def __init__(
        self,
        n_atom_basis: int,
        n_interactions: int,
        radial_basis: nn.Module,
        cutoff_fn: Optional[Callable] = None,
        activation: Optional[Callable] = F.silu,
        max_z: int = 100,
        shared_interactions: bool = False,
        shared_filters: bool = False,
        epsilon: float = 1e-8,
    ):
        """
        Args:
            n_atom_basis: number of features to describe atomic environments.
                This determines the size of each embedding vector; i.e. embeddings_dim.
            n_interactions: number of interaction blocks.
            radial_basis: layer for expanding interatomic distances in a basis set
            cutoff_fn: cutoff function
            activation: activation function
            shared_interactions: if True, share the weights across
                interaction blocks.
            shared_interactions: if True, share the weights across
                filter-generating networks.
            epsilon: stability constant added in norm to prevent numerical instabilities
        """
        super(PaiNN, self).__init__()

        self.n_atom_basis = n_atom_basis
        self.n_interactions = n_interactions
        self.cutoff_fn = cutoff_fn
        self.cutoff = cutoff_fn.cutoff
        self.radial_basis = radial_basis

        self.embedding = nn.Embedding(max_z, n_atom_basis, padding_idx=0)

        self.share_filters = shared_filters

        if shared_filters:
            self.filter_net = snn.Dense(
                self.radial_basis.n_rbf, 3 * n_atom_basis, activation=None
            )
        else:
            self.filter_net = snn.Dense(
                self.radial_basis.n_rbf,
                self.n_interactions * n_atom_basis * 3,
                activation=None,
            )

        self.interactions = snn.replicate_module(
            lambda: PaiNNInteraction(
                n_atom_basis=self.n_atom_basis, activation=activation
            ),
            self.n_interactions,
            shared_interactions,
        )
        self.mixing = snn.replicate_module(
            lambda: PaiNNMixing(
                n_atom_basis=self.n_atom_basis, activation=activation, epsilon=epsilon
            ),
            self.n_interactions,
            shared_interactions,
        )

    def forward(self, inputs: Dict[str, torch.Tensor]):
        """
        Compute atomic representations/embeddings.

        Args:
            inputs (dict of torch.Tensor): SchNetPack dictionary of input tensors.

        Returns:
            torch.Tensor: atom-wise representation.
            list of torch.Tensor: intermediate atom-wise representations, if
            return_intermediate=True was used.
        """
        # get tensors from input dictionary
        atomic_numbers = inputs[properties.Z]
        r_ij = inputs[properties.Rij]
        idx_i = inputs[properties.idx_i]
        idx_j = inputs[properties.idx_j]
        n_atoms = atomic_numbers.shape[0]

        # compute atom and pair features
        d_ij = torch.norm(r_ij, dim=1, keepdim=True)
        dir_ij = r_ij / d_ij
        phi_ij = self.radial_basis(d_ij)
        fcut = self.cutoff_fn(d_ij)

        filters = self.filter_net(phi_ij) * fcut[..., None]
        if self.share_filters:
            filter_list = [filters] * self.n_interactions
        else:
            filter_list = torch.split(filters, 3 * self.n_atom_basis, dim=-1)

        q = self.embedding(atomic_numbers)[:, None]
        qs = q.shape
        mu = torch.zeros((qs[0], 3, qs[2]), device=q.device)

        for i, (interaction, mixing) in enumerate(zip(self.interactions, self.mixing)):
            q, mu = interaction(q, mu, filter_list[i], dir_ij, idx_i, idx_j, n_atoms)
            q, mu = mixing(q, mu)

        q = q.squeeze(1)

        inputs["scalar_representation"] = q
        inputs["vector_representation"] = mu
        return inputs


ModuleNotFoundError: No module named 'schnetpack'